# March Madness 2024 | ML Bracket Prediction
Author Glen Joy (c) 2024

This notebook trains a random-forest model on historical NCAA college basketball statistics to make formulate a predicted bracket for the 2024 season.

In [1]:
import pandas as pd
import numpy as np
import random

## 1. Reading in Training Data
We will incorporate data from a variety of sources. This includes 'traditional' regular season team stats such as winning percentage, 3-pt percentage, free throw percentage, etc. We will also use a team's historic tournament performance. 

Since these are yearly stats, they will be merged on (Team Name, Year) where (Team Name, Year) is set as the multi-index.

Additionally, for funzies, we'll incorporate external 'nontraditional' factors such as public rankings, school spirit, academic ranking, etc.

### Data Source 1: Historic Team Stats

In [2]:
# Reading in historic team season stats data. Has data from 2013 to 2023
df = pd.read_csv('./data/archive1/cbb.csv')
df['WIN_PCT'] = df['W']/df['G']
df.drop(['G', 'W'], axis=1, inplace=True)
df.head()

,TEAM,CONF,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,...,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR,WIN_PCT
0,North Carolina,ACC,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,...,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016,0.825000
1,Wisconsin,B10,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,...,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015,0.900000
2,Michigan,B10,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,...,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018,0.825000
3,Texas Tech,B12,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,...,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019,0.815789
4,Gonzaga,WCC,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,...,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017,0.948718


In [3]:
df.columns

Index(['TEAM', 'CONF', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR',
       'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O', '3P_D',
       'ADJ_T', 'WAB', 'POSTSEASON', 'SEED', 'YEAR', 'WIN_PCT'],
      dtype='object')

### Data Source 2: Historic Matchups

In [4]:
df2 = pd.read_csv('./data/MNCAATourneyCompactResults.csv')
df2['YEAR'] = df2['Season']
df2.drop(['Season'], axis=1, inplace=True)
df2.head()

,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,YEAR
0,136,1116,63,1234,54,N,0,1985
1,136,1120,59,1345,58,N,0,1985
2,136,1207,68,1250,43,N,0,1985
3,136,1229,58,1425,55,N,0,1985
4,136,1242,49,1325,38,N,0,1985


### Team Lookup Table

In [5]:
teamsdf = pd.read_csv('./data/MTeams.csv')
teamsdf.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2024
1,1102,Air Force,1985,2024
2,1103,Akron,1985,2024
3,1104,Alabama,1985,2024
4,1105,Alabama A&M,2000,2024


### Data Source 3: Conference Stats

In [6]:
df3 = pd.read_csv('./data/archive/Conference Stats.csv') # this is regular season only!
df3 = df3[df3['YEAR'] != 2024] # removing imcomplete 2024 data
df3['WIN_PCT_CONF'] = df3['W']/df3['G']
df3.drop(['W', 'G', 'L', 'WIN%', 'CONF ID'], axis=1, inplace=True)
df3.head()

,YEAR,CONF,BADJ EM,BADJ O,BADJ D,BARTHAG,EFG%,EFGD%,FTR,FTRD,...,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB,WIN_PCT_CONF
33,2023,B12,18.0,111.8,93.8,0.883,51.6,48.5,33.5,33.0,...,80.522,2.140,55.266,72.5,71.7,1.059,0.978,34.920,2.9,0.629969
34,2023,B10,14.2,110.4,96.2,0.830,51.1,48.7,29.5,26.6,...,80.919,1.845,55.026,71.1,70.7,1.061,1.001,31.036,0.2,0.585526
35,2023,BE,13.0,110.8,97.8,0.808,51.7,49.9,30.4,28.1,...,80.679,1.935,54.668,73.5,72.7,1.065,1.019,30.790,-1.0,0.561111
36,2023,SEC,12.5,109.2,96.7,0.802,49.7,47.8,34.4,32.4,...,80.909,1.968,47.670,71.1,71.5,1.053,0.998,28.681,-0.8,0.591304
37,2023,P12,11.4,107.9,96.5,0.783,49.9,47.9,31.1,28.8,...,81.439,1.855,55.558,71.7,71.0,1.030,0.991,28.629,-2.2,0.544529


In [7]:
df3.columns

Index(['YEAR', 'CONF', 'BADJ EM', 'BADJ O', 'BADJ D', 'BARTHAG', 'EFG%',
       'EFGD%', 'FTR', 'FTRD', 'TOV%', 'TOV%D', 'OREB%', 'DREB%', 'OP OREB%',
       'OP DREB%', 'RAW T', '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%',
       'AST%', 'OP AST %', '2PTR', '3PTR', '2PTRD', '3PTRD', 'BADJ T',
       'AVG HGT', 'EFF HGT', 'EXP', 'TALENT', 'FT%', 'OP FT%', 'PPPO', 'PPPD',
       'ELITE SOS', 'WAB', 'WIN_PCT_CONF'],
      dtype='object')

## 2. Merging, Fusing, and Deconflicting Datasets
We will combine all of these datasets into a large one to train using.

#### Combining Team and Conference Stats by Year

In [8]:
cdf = pd.merge(df, df3, on=['CONF', 'YEAR'], how='outer') # merging team stats and conference stats

In [9]:
cdf.head()

,TEAM,CONF,ADJOE,ADJDE,BARTHAG_x,EFG_O,EFG_D,TOR,TORD,ORB,...,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB_y,WIN_PCT_CONF
0,NaN,A10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.066,1.819,24.809,70.3,70.0,1.050,1.015,21.975,-4.2,0.561485
1,NaN,A10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.241,1.673,28.691,68.2,68.7,1.026,1.004,19.678,-6.0,0.540230
2,NaN,A10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.307,1.558,29.958,67.7,67.6,1.017,0.995,23.936,-4.5,0.554023
3,NaN,A10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80.413,1.609,30.035,68.4,69.8,1.020,1.003,18.832,-6.2,0.537757
4,NaN,A10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79.954,1.660,28.115,70.2,71.0,1.034,1.004,22.225,-3.5,0.552511


In [10]:
cdf.columns

Index(['TEAM', 'CONF', 'ADJOE', 'ADJDE', 'BARTHAG_x', 'EFG_O', 'EFG_D', 'TOR',
       'TORD', 'ORB', 'DRB', 'FTR_x', 'FTRD_x', '2P_O', '2P_D', '3P_O', '3P_D',
       'ADJ_T', 'WAB_x', 'POSTSEASON', 'SEED', 'YEAR', 'WIN_PCT', 'BADJ EM',
       'BADJ O', 'BADJ D', 'BARTHAG_y', 'EFG%', 'EFGD%', 'FTR_y', 'FTRD_y',
       'TOV%', 'TOV%D', 'OREB%', 'DREB%', 'OP OREB%', 'OP DREB%', 'RAW T',
       '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%', 'AST%', 'OP AST %',
       '2PTR', '3PTR', '2PTRD', '3PTRD', 'BADJ T', 'AVG HGT', 'EFF HGT', 'EXP',
       'TALENT', 'FT%', 'OP FT%', 'PPPO', 'PPPD', 'ELITE SOS', 'WAB_y',
       'WIN_PCT_CONF'],
      dtype='object')

In [11]:
cdf = cdf[cdf['TEAM'].notna()] # dropping rows where we have conference data but no corresponding team
cdf.head()

,TEAM,CONF,ADJOE,ADJDE,BARTHAG_x,EFG_O,EFG_D,TOR,TORD,ORB,...,EFF HGT,EXP,TALENT,FT%,OP FT%,PPPO,PPPD,ELITE SOS,WAB_y,WIN_PCT_CONF
5,George Washington,A10,98.8,93.0,0.6650,46.6,47.4,23.4,21.3,38.2,...,79.886,1.705,29.413,69.6,69.9,1.033,1.0,23.936,-3.4,0.567134
6,St. Bonaventure,A10,107.9,102.5,0.6442,51.6,51.6,20.0,21.5,32.3,...,79.886,1.705,29.413,69.6,69.9,1.033,1.0,23.936,-3.4,0.567134
7,Dayton,A10,111.8,99.4,0.7951,53.0,49.2,21.7,21.0,35.2,...,79.886,1.705,29.413,69.6,69.9,1.033,1.0,23.936,-3.4,0.567134
8,Xavier,A10,105.1,94.3,0.7765,50.1,46.7,20.4,18.6,34.3,...,79.886,1.705,29.413,69.6,69.9,1.033,1.0,23.936,-3.4,0.567134
9,Saint Joseph's,A10,107.4,96.8,0.7681,50.6,46.1,18.7,16.2,31.7,...,79.886,1.705,29.413,69.6,69.9,1.033,1.0,23.936,-3.4,0.567134


In [12]:
cdf.size

218426

#### Embelishing Team/Conference Stats with Historic Matchups

In [13]:
df2 = df2[df2['YEAR'] >= 2008] # looking at 2008 and up since thats when all our data overlaps

In [14]:
df2.head()

,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,YEAR
1456,134,1291,69,1164,60,N,0,2008
1457,136,1181,71,1125,70,N,0,2008
1458,136,1242,85,1340,61,N,0,2008
1459,136,1243,80,1425,67,N,0,2008
1460,136,1266,74,1246,66,N,0,2008


In [15]:
teamsdf.set_index('TeamID', inplace=True)

In [16]:
teamsdf.head()

,TeamName,FirstD1Season,LastD1Season
TeamID,,,
1101,Abilene Chr,2014,2024
1102,Air Force,1985,2024
1103,Akron,1985,2024
1104,Alabama,1985,2024
1105,Alabama A&M,2000,2024


In [17]:
df2.head()

,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,YEAR
1456,134,1291,69,1164,60,N,0,2008
1457,136,1181,71,1125,70,N,0,2008
1458,136,1242,85,1340,61,N,0,2008
1459,136,1243,80,1425,67,N,0,2008
1460,136,1266,74,1246,66,N,0,2008


In [18]:
# Looking up and replacing team ID with string name
df2['WTEAM'] = df2.apply(lambda row : teamsdf.loc[row['WTeamID']]['TeamName'], axis=1)

df2['LTEAM'] = df2.apply(lambda row : teamsdf.loc[row['LTeamID']]['TeamName'], axis=1)

df2['TEAM_1'] = df2.apply(lambda row : random.choice([row['WTEAM'], row['LTEAM']]), axis=1)
df2['TEAM_2'] = df2.apply(lambda row : row['LTEAM'] if row['TEAM_1'] == row['WTEAM'] else row['WTEAM'], axis=1)

df2['WINNER'] = df2.apply(lambda row : 1 if row['TEAM_1'] == row['WTEAM'] else 2, axis=1)

In [19]:
df2.columns

Index(['DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT',
       'YEAR', 'WTEAM', 'LTEAM', 'TEAM_1', 'TEAM_2', 'WINNER'],
      dtype='object')

In [20]:
df2.drop(['DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WTEAM', 'LTEAM'], axis=1, inplace=True)

In [21]:
cdf.YEAR.min()

2013

In [22]:
df2 = df2[df2['YEAR'] >= 2013]

In [23]:
cdf['TEAM'] = cdf['TEAM'].apply(lambda name : name.strip('.'))

In [24]:
df2.shape

(669, 4)

In [25]:
tdfs = []
for year in df2['YEAR'].unique():
    df_temp = df2[df2['YEAR'] == year].copy()
    cdf_temp = cdf[cdf['YEAR'] == year].copy()
    df_temp = df_temp.merge(cdf_temp, left_on='TEAM_1', right_on='TEAM', suffixes=['_ONE', '_TWO'])
    df_temp = df_temp.merge(cdf_temp, left_on='TEAM_2', right_on='TEAM', suffixes=['_ONE', '_TWO'])
    tdfs.append(df_temp.copy())
traindf = pd.concat(tdfs)

In [26]:
traindf.head()

,YEAR_ONE,TEAM_1,TEAM_2,WINNER,TEAM_ONE,CONF_ONE,ADJOE_ONE,ADJDE_ONE,BARTHAG_x_ONE,EFG_O_ONE,...,EFF HGT_TWO,EXP_TWO,TALENT_TWO,FT%_TWO,OP FT%_TWO,PPPO_TWO,PPPD_TWO,ELITE SOS_TWO,WAB_y_TWO,WIN_PCT_CONF_TWO
0,2013,LIU Brooklyn,James Madison,2,LIU Brooklyn,NEC,108.1,111.2,0.4195,54.4,...,79.880,1.595,19.579,69.9,70.3,1.005,1.029,12.956,-11.2,0.448680
1,2013,La Salle,Boise St,1,La Salle,A10,112.0,96.2,0.8516,51.9,...,80.035,1.826,24.940,70.8,68.8,1.031,0.980,25.212,-1.4,0.611722
2,2013,Belmont,Arizona,2,Belmont,OVC,108.9,96.2,0.8064,56.5,...,81.109,1.664,55.458,70.3,69.5,1.027,0.977,26.463,-1.8,0.587629
3,2013,Butler,Bucknell,1,Butler,A10,109.2,93.0,0.8624,50.7,...,79.883,1.667,3.239,72.4,68.8,1.008,1.016,14.044,-9.8,0.485830
4,2013,California,UNLV,1,California,P12,105.4,92.3,0.8211,48.0,...,80.035,1.826,24.940,70.8,68.8,1.031,0.980,25.212,-1.4,0.611722


In [27]:
new_name = 'YEAR'
traindf = traindf.rename(columns={'YEAR_ONE': new_name})

In [28]:
traindf.shape

(538, 128)

## 2. Training

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [30]:
labels = np.array(traindf['WINNER'])

In [31]:
traindf = traindf.drop(['WINNER', 'TEAM_1', 'TEAM_2', 'TEAM_ONE', 'TEAM_TWO', 'CONF_ONE', 'CONF_TWO', 'POSTSEASON_ONE', 'POSTSEASON_TWO'], axis=1) 

In [32]:
# splitting our data into training data and test data, specifically using 25% for testing
train_features, test_features, train_labels, test_labels = train_test_split(traindf, labels, test_size = 0.20, random_state = 42)

In [33]:
# Instantiate model with 5000 decision trees
rf = RandomForestClassifier(n_estimators = 5000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [34]:
predictions = rf.predict(test_features)

In [35]:
# Checking Accuracy
print("Accuracy:",metrics.accuracy_score(test_labels, predictions))

Accuracy: 0.6851851851851852


In [36]:
traindf.head()

,YEAR,ADJOE_ONE,ADJDE_ONE,BARTHAG_x_ONE,EFG_O_ONE,EFG_D_ONE,TOR_ONE,TORD_ONE,ORB_ONE,DRB_ONE,...,EFF HGT_TWO,EXP_TWO,TALENT_TWO,FT%_TWO,OP FT%_TWO,PPPO_TWO,PPPD_TWO,ELITE SOS_TWO,WAB_y_TWO,WIN_PCT_CONF_TWO
0,2013,108.1,111.2,0.4195,54.4,52.6,20.3,18.4,35.2,33.5,...,79.880,1.595,19.579,69.9,70.3,1.005,1.029,12.956,-11.2,0.448680
1,2013,112.0,96.2,0.8516,51.9,49.3,17.1,21.3,29.0,34.2,...,80.035,1.826,24.940,70.8,68.8,1.031,0.980,25.212,-1.4,0.611722
2,2013,108.9,96.2,0.8064,56.5,48.0,19.6,25.0,29.7,34.2,...,81.109,1.664,55.458,70.3,69.5,1.027,0.977,26.463,-1.8,0.587629
3,2013,109.2,93.0,0.8624,50.7,46.7,20.2,17.4,35.9,26.3,...,79.883,1.667,3.239,72.4,68.8,1.008,1.016,14.044,-9.8,0.485830
4,2013,105.4,92.3,0.8211,48.0,44.3,18.9,16.8,32.5,30.7,...,80.035,1.826,24.940,70.8,68.8,1.031,0.980,25.212,-1.4,0.611722


## 3. Running Predictions

In [37]:
# reading in 2024 seasonal data
df24 = pd.read_csv('')

FileNotFoundError: [Errno 2] No such file or directory: ''